# Computation Method

The SN² Solver has two "equivalent" computational methods.
These two methods result in the same weight partial derivative in each step.
However, you might computationally prefer one method above the other.

The two computational methods of the SN² Solver are:
- the covariance method, and
- the weight accumulation method.
By default, the SN² Solver uses the covariance method. However, one might use the accumulation method at will.

Let's get started with the same example as in [introduction](introduction.ipynb).

In [1]:
import torch
from sn2_cuda import SN2Solver

device = torch.device("cuda")

struct = torch.tensor([
        [1, 1, 1, 0],
        [0, 1, 0, 1],
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
        [0, 1, 1, 1],  # V_X
        [0, 0, 1, 0],  # V_BP
        [0, 0, 0, 1],  # V_BMI
        [0, 0, 0, 0],  # V_Y
    ], dtype=torch.bool, device=device)

sample_covariance = torch.tensor([
        [2,  3,  6,  8],
        [3,  7, 12, 16],
        [6, 12, 23, 30],
        [8, 16, 30, 41],
    ], device=device)

We may initialize the solver with the accumulation method, instead of the usual method.

In [2]:
pmDAG = SN2Solver(
    struct, 
    sample_covariance=sample_covariance, 
    method=SN2Solver.METHODS.ACCUM
)

Note that we set `method=sc.SEMNANSolver.METHODS.ACCUM` in the initialization. Alternatively, we could use `method=sc.SEMNANSolver.METHODS.COVAR`.
Now, we can start the computation using the same algorithm as in [introduction](introduction.ipynb).

In [3]:
max_iterations = 10000
min_error = 1.0e-7
optim = torch.optim.Adamax([pmDAG.weights], lr=0.001)

for i in range(max_iterations):
    pmDAG.forward()
    error = pmDAG.loss().item()

    if error < min_error:
        break

    pmDAG.backward()
    optim.step()

    if i % (max_iterations / 10) == 0:
        print(f"iteration={i:<10} loss={error:<15.5}")
else:
    print("Did not converge in the maximum number of iterations!")

iteration=0          loss=8.0531         
iteration=1000       loss=1.4745         
iteration=2000       loss=0.63672        
iteration=3000       loss=0.040596       
iteration=4000       loss=1.3947e-05     


The resulting visible covariance matrix is:

In [4]:
print(pmDAG.visible_covariance_)

tensor([[ 2.0005,  3.0001,  6.0007,  8.0005],
        [ 3.0001,  6.9994, 11.9997, 15.9989],
        [ 6.0007, 11.9997, 23.0007, 30.0013],
        [ 8.0005, 15.9989, 30.0013, 41.0006]], device='cuda:0')
